In [1]:
# !pip install langchain_community
# ! pip install langchain_openai

In [2]:
import pandas as pd
import json
import traceback
import os

In [3]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.

True

In [4]:
OPENAI_KEY = os.getenv("OPENAI_KEY")

In [6]:
# import openai
# import langchain
# from langchain.chat_models import ChatOpenAI

In [7]:
import openai
import langchain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(openai_api_key=OPENAI_KEY, model="gpt-4o-mini", temperature=0.5)

In [8]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [9]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [10]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in difficulty level - {tone} 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [11]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\yashd\AppData\Local\Temp\ipykernel_24040\320620758.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [13]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True)

In [14]:
TEXT = """Electromagnetic induction is the process of generating an electromotive force (EMF) or voltage across a conductor when it is exposed to a changing magnetic field. This fundamental principle was discovered by Michael Faraday in 1831 and is the basis for many electrical devices, including transformers, generators, and induction cooktops. According to Faraday’s Law, the magnitude of the induced EMF is directly proportional to the rate of change of magnetic flux through the conductor. Mathematically, it is expressed as:

    𝓔 = -dΦB/dt

where 𝓔 is the induced EMF, and ΦB is the magnetic flux. The negative sign represents Lenz’s Law, which states that the induced current will always oppose the change in magnetic flux that caused it. This opposition is a consequence of the law of conservation of energy, ensuring that energy is neither created nor destroyed.

There are two main types of electromagnetic induction: mutual induction and self-induction. Mutual induction occurs when a changing current in one coil induces an EMF in a nearby coil, as seen in transformers. Self-induction, on the other hand, occurs when a changing current in a coil induces an EMF in the same coil due to its own changing magnetic field. The inductance (L) of a coil determines how effectively it can induce an EMF in response to changing current, and its unit is the Henry (H).

This principle has widespread applications in everyday life. In power plants, electrical generators use electromagnetic induction to convert mechanical energy into electrical energy. In wireless charging technology, coils in the charger and the device create an induced current to charge batteries without physical contact. Even credit card readers use electromagnetic induction to extract information from magnetic strips. The concept remains a cornerstone of modern electrical engineering, making it one of the most crucial discoveries in physics."""


In [15]:
NUMBER=5 
SUBJECT="Electromagnetic Induction"
TONE="simple"

In [16]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\yashd\AppData\Local\Temp\ipykernel_24040\2654919708.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=generate_evaluate_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Electromagnetic induction is the process of generating an electromotive force (EMF) or voltage across a conductor when it is exposed to a changing magnetic field. This fundamental principle was discovered by Michael Faraday in 1831 and is the basis for many electrical devices, including transformers, generators, and induction cooktops. According to Faraday’s Law, the magnitude of the induced EMF is directly proportional to the rate of change of magnetic flux through the conductor. Mathematically, it is expressed as:

    𝓔 = -dΦB/dt

where 𝓔 is the induced EMF, and ΦB is the magnetic flux. The negative sign represents Lenz’s Law, which states that the induced current will always oppose the change in magnetic flux that caused it. This opposition is a consequence of the law of conservation of energy, ensuring that energy is neither created nor destroyed.

There are two main types o

In [17]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2000
Prompt Tokens:1105
Completion Tokens:895
Total Cost:0.0007027499999999999


In [18]:
response

{'text': 'Electromagnetic induction is the process of generating an electromotive force (EMF) or voltage across a conductor when it is exposed to a changing magnetic field. This fundamental principle was discovered by Michael Faraday in 1831 and is the basis for many electrical devices, including transformers, generators, and induction cooktops. According to Faraday’s Law, the magnitude of the induced EMF is directly proportional to the rate of change of magnetic flux through the conductor. Mathematically, it is expressed as:\n\n    𝓔 = -dΦB/dt\n\nwhere 𝓔 is the induced EMF, and ΦB is the magnetic flux. The negative sign represents Lenz’s Law, which states that the induced current will always oppose the change in magnetic flux that caused it. This opposition is a consequence of the law of conservation of energy, ensuring that energy is neither created nor destroyed.\n\nThere are two main types of electromagnetic induction: mutual induction and self-induction. Mutual induction occurs wh

In [19]:
quiz=response.get("quiz")
quiz=json.loads(quiz)
quiz

{'1': {'mcq': 'Who discovered the principle of electromagnetic induction?',
  'options': {'a': 'Nikola Tesla',
   'b': 'Michael Faraday',
   'c': 'James Clerk Maxwell',
   'd': 'Albert Einstein'},
  'correct': 'b'},
 '2': {'mcq': "What does Faraday's Law state about the induced electromotive force (EMF)?",
  'options': {'a': 'It is inversely proportional to the rate of change of magnetic flux.',
   'b': 'It is directly proportional to the rate of change of magnetic flux.',
   'c': 'It is constant regardless of magnetic flux changes.',
   'd': 'It depends solely on the resistance of the conductor.'},
  'correct': 'b'},
 '3': {'mcq': 'What is the unit of inductance?',
  'options': {'a': 'Volt', 'b': 'Ampere', 'c': 'Henry', 'd': 'Ohm'},
  'correct': 'c'},
 '4': {'mcq': 'Which type of electromagnetic induction occurs when a changing current in one coil induces an EMF in a nearby coil?',
  'options': {'a': 'Self-induction',
   'b': 'Mutual induction',
   'c': 'Electromagnetic induction',
  

In [20]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})
quiz_table_data

[{'MCQ': 'Who discovered the principle of electromagnetic induction?',
  'Choices': 'a: Nikola Tesla | b: Michael Faraday | c: James Clerk Maxwell | d: Albert Einstein',
  'Correct': 'b'},
 {'MCQ': "What does Faraday's Law state about the induced electromotive force (EMF)?",
  'Choices': 'a: It is inversely proportional to the rate of change of magnetic flux. | b: It is directly proportional to the rate of change of magnetic flux. | c: It is constant regardless of magnetic flux changes. | d: It depends solely on the resistance of the conductor.',
  'Correct': 'b'},
 {'MCQ': 'What is the unit of inductance?',
  'Choices': 'a: Volt | b: Ampere | c: Henry | d: Ohm',
  'Correct': 'c'},
 {'MCQ': 'Which type of electromagnetic induction occurs when a changing current in one coil induces an EMF in a nearby coil?',
  'Choices': 'a: Self-induction | b: Mutual induction | c: Electromagnetic induction | d: Static induction',
  'Correct': 'b'},
 {'MCQ': 'What principle ensures that the induced cur

In [21]:
quizDataFrame=pd.DataFrame(quiz_table_data)
quizDataFrame

,MCQ,Choices,Correct
0,Who discovered the principle of electromagneti...,a: Nikola Tesla | b: Michael Faraday | c: Jame...,b
1,What does Faraday's Law state about the induce...,a: It is inversely proportional to the rate of...,b
2,What is the unit of inductance?,a: Volt | b: Ampere | c: Henry | d: Ohm,c
3,Which type of electromagnetic induction occurs...,a: Self-induction | b: Mutual induction | c: E...,b
4,What principle ensures that the induced curren...,a: Faraday’s Law | b: Lenz’s Law | c: Ohm’s La...,b
